In [2]:
import numpy as np

In [ ]:
def meta_mle(beta,gamma,sol,time_list,tmax,kernel,distances,causes):
    N = len(sol[0])
    L1 = 0
    node_list = np.arange(0,N)
    for j in range(N): #iterate for each node
        t = 0
        for i in range(0, len(time_list) - 1):
            if ((sol[i+1][j][1] == sol[i][j][1]) and (sol[i+1][j][0] == sol[i][j][0])):
                pass # do nothing if the event will be counted later
            elif (sol[i + 1][j][1] - sol[i][j][1]) > 0: # if infection

                #calculate exponential term: need to iterate over every possible event! Many are possible alas.
                expo = 0
                #add domestic events
                for node in node_list:
                    expo += (beta) * sol[i][node][0] * sol[i][node][1] + gamma * sol[i][node][1]
                # add foreign infections
                for node in node_list:
                    foreign_nodes = np.delete(node_list,node)
                    for foreign_n in foreign_nodes:
                        expo += kernel((beta) * sol[i][node][0] * sol[i][foreign_n][1],distances[node,foreign_n])

                expo = -expo*(time_list[i+1]-time_list[i])
                if causes[i] == "d":
                    pre_expo = np.log((beta) * sol[i][j][0] * sol[i][j][1])
                else:
                    pre_expo = np.log(kernel((beta) * sol[i][j][0] * sol[i][causes[i]-1][1],distances[j,causes[i]-1]))

                L1 += (pre_expo+ expo) #infection
            else: #if recovery
                pre_expo = gamma * sol[i][j][1]
                expo = 0
                #add domestic events
                for node in node_list:
                    expo += (beta) * sol[i][node][0] * sol[i][node][1] + gamma * sol[i][node][1]
                # add foreign infections
                for node in node_list:
                    foreign_nodes = np.delete(node_list,node)
                    for foreign_n in foreign_nodes:
                        expo += kernel((beta) * sol[i][node][0] * sol[i][foreign_n][1],distances[node,foreign_n])
                expo = -expo*(time_list[i+1]-time_list[i])
                L1 += (np.log(pre_expo) + expo)
        L2 = 0
    return L1 + L2

In [75]:
hand_calc = 2*1*1*np.exp(-9) * 2 * np.exp(-16)
print(np.log(hand_calc))

-23.61370563888011


In [72]:
solution = [[[1,1,0],[2,0,0]],[[1,1,0],[1,1,0]],[[1,0,1],[1,1,0]]]
times = [0,1,2]
def kernel(x,d):
    return x/5
distances = np.array([[1,1],[1,1]])
causes = [1,"d"]

In [73]:
test_answer = meta_mle(5,2,solution,times,100,kernel,distances,causes)

In [74]:
print(test_answer)

-23.61370563888011
